In [ ]:
import json
import os
from typing import List

from continuedev.core.sdk import ContinueSDK
from continuedev.libs.index.chunkers import Chunk
from continuedev.headless import start_headless_session
from continuedev.libs.index.pipelines.main import main_retrieval_pipeline

session = await start_headless_session(directory="/Users/natesesti/Desktop/continue/server/continuedev")
sdk: ContinueSDK = session.autopilot.continue_sdk

/Users/natesesti/Desktop/continue/server/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-10-24 20:12:10,353] [DEBUG] New session: None
[2023-10-24 20:12:10,359] [DEBUG] Loaded Continue config file from /Users/natesesti/.continue/config.py
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x139471a90>
[2023-10-24 20:12:10,416] [DEBUG] Starting context manager


[2023-10-24 20:12:10,465] [DEBUG] Loading Meilisearch index...
[2023-10-24 20:12:10,560] [DEBUG] Loaded 0 documents into meilisearch in 0.008828163146972656 seconds for context provider code
[2023-10-24 20:12:10,569] [DEBUG] Loaded 1 documents into meilisearch in 0.018381834030151367 seconds for context provider search
[2023-10-24 20:12:10,580] [DEBUG] Loaded 1 documents into meilisearch in 0.029497146606445312 seconds for context provider url
[2023-10-24 20:12:10,588] [DEBUG] Loaded 1 documents into meilisearch in 0.03736305236816406 seconds for context provider terminal
[2023-10-24 20:12:10,594] [DEBUG] Loaded 1 documents into meilisearch in 0.04324007034301758 seconds for context provider diff
[2023-10-24 20:12:10,597] [DEBUG] Loaded 150 documents into meilisearch in 0.04632210731506348 seconds for context provider file
[2023-10-24 20:12:10,599] [DEBUG] Loaded Meilisearch index


In [ ]:
print(sdk.models.summarize)

async def f(q: str):
    return await main_retrieval_pipeline(q, sdk)

In [4]:
with open("y.json", "r") as file:
    data = json.load(file)

print(data)
questions = data["questions"]
base_path = data["basePath"]

def abs_paths(question):
    return list(map(
        lambda x: os.path.join(base_path, x),
        question["files"]
    ))

def in_y(chunk: Chunk, question):
    return chunk.document_id in abs_paths(question)

def f1_score(chunks: List[Chunk], question) -> float:
    chosen_files = list(set(map(lambda c: c.document_id, chunks)))
    actual_files = abs_paths(question)
    tp = len(list(filter(lambda x: x in actual_files, chosen_files)))
    fp = len(chosen_files) - tp
    fn = len(actual_files) - tp
    if tp == 0:
        return 0

    return 2 * tp / (2 * tp + fp + fn)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13948a010>


{'basePath': '/Users/natesesti/Desktop/continue/server/continuedev', 'questions': [{'q': 'How does the Continue FastAPI server work?', 'files': ['server/main.py', 'server/gui.py', 'server/ide.py']}, {'q': 'Where is logic for pruning the prompt tokens?', 'files': ['libs/util/count_tokens.py', 'libs/llm/base.py']}, {'q': 'Where does codebase indexing happen?', 'files': ['plugins/steps/chroma.py', 'libs/index/indices/chroma_index.py', 'libs/index/indices/meilisearch_index.py', 'libs/index/indices/base.py', 'core/context.py']}, {'q': 'How is meilisearch used?', 'files': ['plugins/steps/chroma.py', 'libs/index/indices/meilisearch_index.py', 'libs/index/indices/base.py', 'core/context.py', 'server/meilisearch_server.py']}]}


In [6]:
scores = []
for question in questions:
    chunks = await f(question["q"])
    score = f1_score(chunks, question)
    print(score)
    scores.append(score)

print(scores)
print(sum(scores) / len(scores))

Scanning 50 files...
Selecting most important files...
0.16666666666666666
Scanning 50 files...
Selecting most important files...
0.4444444444444444
Scanning 50 files...
Selecting most important files...
0.5
Scanning 50 files...
Selecting most important files...
0.5454545454545454
[0.16666666666666666, 0.4444444444444444, 0.5, 0.5454545454545454]
0.41414141414141414
